In [1]:
import os
import re
import random
import json
import uuid
import praw
import pathlib
import textwrap
# import numpy as np
from IPython.display import display, Markdown, Latex
from pprint import pprint
import ast

from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, WebsiteSearchTool, DirectoryReadTool, FileReadTool
from crewai_tools import BaseTool, MDXSearchTool
from pydantic import BaseModel

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
api_keys = {
    "openai": "openai_api_key",
    "ollama": "NA",
    # Add more keys if needed
}

In [15]:
# Specify the filename for the JSON file
# json_file_path = 'api_keys.json'

# # Write the dictionary to a JSON file
# with open(json_file_path, 'w') as json_file:
#     json.dump(api_keys, json_file, indent=4)

# print(f"JSON file created successfully at: {json_file_path}")

In [24]:
api_file_path = 'api_keys.json'
with open(json_file_path, 'r') as file:
    api_data = json.load(file)   
print(api_data['ollama'], api_data['praw']['client_id'])

NA a0wBU22OWImc8J2CRnYFDQ


In [7]:
import yaml

gemini_generated_keywords = [
    "Interior Design Automation",
    "Virtual Room Redesign",
    "AI Home Decor Suggestions",
    "Furniture Arrangement AI",
    "Room Style Transformation"
]


user_provided_keywords = [
    "Eco-friendly Interior Suggestions",
    "Online Interior Design Tools",
    "Home Renovation App",
    "Modern Home Styling AI",
    "Luxury Home Design Automation"
]

broad_keywords = [
      "interior design",
      "home decor",
      "furniture arrangement",
      "room makeover",
      "style transformation",
      "virtual design",
      "home improvement"
]

casa_ai_descr = "Casa AI is the next generation tool for transforming interior spaces. Designed for homeowners, interior designers, \
and real estate professionals, our app uses advanced AI to reimagine any room based on your style and functional needs."

# Define the data to be written to the YAML file
config_data = {
    "gemini_generated_keywords": gemini_generated_keywords,
    "user_provided_keywords": user_provided_keywords,
    "broad_keywords": broad_keywords,
    "product_description": casa_ai_descr
}

# Specify the filename for the YAML file
yaml_file_path = 'casaai_config.yaml'

# Write the data to a YAML file
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(config_data, yaml_file, default_flow_style=False, sort_keys=False)

print(f"YAML file created successfully at: {yaml_file_path}")

YAML file created successfully at: casaai_config.yaml


In [10]:
# Specify the YAML file path
yaml_file_path = 'casaai_config.yaml'

# Read the YAML file
with open(yaml_file_path, 'r') as yaml_file:
    config_data = yaml.safe_load(yaml_file)

# Access the data
gemini_generated_keywords = config_data.get('gemini_generated_keywords', [])
user_provided_keywords = config_data.get('user_provided_keywords', [])
broad_keywords = config_data.get('broad_keywords', [])
product_description = config_data.get('product_description', "")

# Print the data to verify
print("Generated Keywords:", gemini_generated_keywords)
print("user_provided Keywords:", user_provided_keywords)
print("Broad Keywords:", broad_keywords)
print("Product Description:", product_description)


Generated Keywords: ['Interior Design Automation', 'Virtual Room Redesign', 'AI Home Decor Suggestions', 'Furniture Arrangement AI', 'Room Style Transformation']
user_provided Keywords: ['Eco-friendly Interior Suggestions', 'Online Interior Design Tools', 'Home Renovation App', 'Modern Home Styling AI', 'Luxury Home Design Automation']
Broad Keywords: ['interior design', 'home decor', 'furniture arrangement', 'room makeover', 'style transformation', 'virtual design', 'home improvement']
Product Description: Casa AI is the next generation tool for transforming interior spaces. Designed for homeowners, interior designers, and real estate professionals, our app uses advanced AI to reimagine any room based on your style and functional needs.


In [25]:
reddit = praw.Reddit(client_id=api_data['praw']['client_id'],
                    client_secret=api_data['praw']['client_secret'],
                    user_agent=api_data['praw']['user_agent'],
                    username=api_data['praw']['username'],
                    check_for_async=False) 

In [26]:
def search_subreddits(keyword):
    subreddits = []
    for subreddit in reddit.subreddits.search_by_name(keyword, exact=False):
        subreddits.append(subreddit.display_name)
    return subreddits

In [27]:
def search_posts(subreddit_name, keyword):
    posts = []
    post_ids = []
    subreddit = reddit.subreddit(subreddit_name)
    for post in subreddit.search(keyword, limit=10):
        post_data = {
            'title': post.title,
            "comment_id": post.id,
            'url': post.url,
            'score': post.score,
            'num_comments': post.num_comments,
            'Post_views': post.view_count,
            'upvote_ratio': post.upvote_ratio,
            'author': str(post.author),
            'created_utc': post.created_utc,
            'image_urls': [],
            'comments': []
        }

        # Check for images in the post
        if hasattr(post, 'url') and (post.url.endswith('.jpg') or post.url.endswith('.png')):
            post_data['image_urls'].append(post.url)
        elif hasattr(post, 'media_metadata'):
            for item_id in post.media_metadata:
                media_item = post.media_metadata[item_id]
                if 'm' in media_item and 'image' in media_item['m']:
                    url = media_item.get('s', {}).get('u', None)
                    if url:
                        post_data['image_urls'].append(url)

        # Fetching comments
        post.comments.replace_more(limit=0)
        for comment in post.comments.list():
            comment_data = {
                "comment_id": comment.id,
                "parent_id": comment.parent_id.split('_')[1],
                "text": comment.body,
                "author": str(comment.author),
                "score": comment.score,
                "created_utc": comment.created_utc,
                "image_url": ""
            }

            # Check for images in comments if applicable
            if hasattr(comment, 'body_html') and 'img src="' in comment.body_html:
                start_index = comment.body_html.find('img src="') + len('img src="')
                end_index = comment.body_html.find('"', start_index)
                comment_data['image_url'] = comment.body_html[start_index:end_index]

            post_data['comments'].append(comment_data)

        posts.append(post_data)
        post_ids.append(post.id)
    
    return posts, post_ids

In [28]:
found_subreddits = [search_subreddits(keyword) for keyword in broad_keywords]
all_subreddits = []
for each in found_subreddits:
    all_subreddits.extend(each)
all_subreddits = set(all_subreddits)
print(len(all_subreddits))

47


In [29]:
reddit_posts = {}
reddit_post_ids = []

for subreddit_name in all_subreddits:
    reddit_posts[subreddit_name] = {}
    for keyword in gemini_generated_keywords:
        posts, post_ids = search_posts(subreddit_name, keyword)    
        reddit_posts[subreddit_name][keyword] = posts
        reddit_post_ids.append(post_ids)

print(len(reddit_posts), len(reddit_post_ids))

47 235


In [35]:
reddit_post_ids_t1 = []
post_ids_t1 = []
post_ids_t2 = [1, 2, 3]
post_ids_t3 = []
reddit_post_ids_t1.extend(post_ids_t1)
print(reddit_post_ids_t1)
reddit_post_ids_t1.extend(post_ids_t2)
print(reddit_post_ids_t1)
reddit_post_ids_t1.extend(post_ids_t3)
print(reddit_post_ids_t1)

[]
[1, 2, 3]
[1, 2, 3]


In [41]:
reddit_post_ids_2 = []
tot_len = 0
for p_id_lst in reddit_post_ids:
    tot_len += len(p_id_lst)
    for p_id in p_id_lst:
        reddit_post_ids_2.append(p_id)
print(len(reddit_post_ids_2), tot_len)

710 710


In [42]:
reddit_post_ids_2

['w7xg8y',
 'spzzua',
 '15b1d1v',
 't1xp1d',
 '11rmlhd',
 'sxdx90',
 '1dgq3v9',
 'vp6fxb',
 'imyu5q',
 'neu3ww',
 '11ke6n7',
 'bt6iz7',
 'kvtxew',
 '6wn933',
 '1ap1xus',
 '1e4obus',
 '1cyurvr',
 'ie0on0',
 '12llljf',
 'uhd570',
 '15rwv19',
 's75c5g',
 '1cn07wf',
 'vp6fxb',
 '142qyab',
 'fw3odd',
 'fjnxw9',
 '18q344o',
 'xfuyhk',
 'fbbxvy',
 '103zq3z',
 'tk6cg2',
 't8rc6g',
 '11k04v7',
 '1auagpm',
 '1bmofj2',
 '1165jg8',
 '1202q68',
 'k6jvv6',
 'oebsqt',
 '1627ecr',
 'hfm1gh',
 '1d87mib',
 '1d9pzud',
 '1cnxzpa',
 '2ovzu4',
 '1c76ih4',
 '189lpy1',
 '16x3wup',
 '145z3nq',
 'z9vq5m',
 'gket0v',
 '16t71ym',
 '170vm2v',
 'wztm1',
 '1678nba',
 '1673ay',
 '1dytjs9',
 '1kip1l',
 '16w84a6',
 '1dicput',
 '1elo4kc',
 '17d2al6',
 '1c7jdc4',
 '1ch58bg',
 '105bwh0',
 '1d5axrw',
 '192w7o1',
 '1cpwlwa',
 '178xljj',
 'kgesxn',
 'idnjjw',
 '1baop85',
 'hq7t1j',
 '18rhobh',
 '18esekk',
 '18e8y88',
 'jbo4wm',
 '1d9y8mk',
 'mmsi20',
 'b003x5',
 '5zyjoa',
 '74m4lz',
 '84ttnr',
 '7y7a33',
 'fa8i4c',
 'cd2udd'

In [ ]:
# reddit_post_ids = reddit_post_ids_2

In [55]:
condensed_data = []
unique_post_ids = set()
unique_comment_ids = set()
dup_post_cnt = 0
unq_post_cnt = 0
dup_comm_cnt = 0
unq_comm_cnt = 0
tot_post_cnt = 0
tot_comm_cnt = 0

cond_data_cnt = 0

In [56]:
unq_reddit_post_ids = set()
for p_id in reddit_post_ids_2:
    unq_reddit_post_ids.add(p_id)
len(unq_reddit_post_ids)

686

In [57]:
for subreddit_name, keywords_posts in reddit_posts.items():
    for keyword, posts in keywords_posts.items():
        for post in posts:
            tot_post_cnt += 1
            if post['comment_id'] in unique_post_ids:
                dup_post_cnt += 1
            else:
                unq_post_cnt += 1
                post_comments = [{
                    'comment_id': post['comment_id'],
                    'parent_id': None,  # The main post has no parent
                    'text': post['title'],
                    'author': post['author'],
                    'score': post['score'],
                }]
                unique_post_ids.add(post['comment_id'])
                for comment in post['comments']:
                    tot_comm_cnt += 1
                    if comment['comment_id'] in unique_comment_ids:
                        dup_comm_cnt += 1                        
                    else:
                        unq_comm_cnt += 1
                        comment_data = {
                            'comment_id': comment['comment_id'],
                            'parent_id': comment['parent_id'],
                            'text': comment['text'],
                            'author': comment['author'],
                            'score': comment['score'],
                        }                    
                        post_comments.append(comment_data)
                        unique_comment_ids.add(comment['comment_id'])
                condensed_data.append(post_comments)

for lst in condensed_data:
    cond_data_cnt += len(lst)

print(f'post_cnt: {dup_post_cnt} + {unq_post_cnt} = {tot_post_cnt}')
print(f'comm_cnt: {dup_comm_cnt} + {unq_comm_cnt} = {tot_comm_cnt}')
print(f'cond_cnt: {cond_data_cnt} = {unq_post_cnt} + {unq_comm_cnt}')
print(f'Cross_ck: {len(unq_reddit_post_ids)} = {unq_post_cnt}')

post_cnt: 24 + 686 = 710
comm_cnt: 0 + 16442 = 16442
cond_cnt: 17128 = 686 + 16442
Cross_ck: 686 = 686


In [58]:
len(condensed_data)

686

In [61]:
condensed_data[0]

[{'comment_id': 'w7xg8y',
  'parent_id': None,
  'text': 'Interior design',
  'author': 'Ainzi-RS',
  'score': 2},
 {'comment_id': 'ihmgyqq',
  'parent_id': 'w7xg8y',
  'text': "There was an IKEA app back when the Vive first came out. No idea if it is still a thing, but it's definitely a good idea...",
  'author': 'TheKramer89',
  'score': 2},
 {'comment_id': 'ihma1i4',
  'parent_id': 'w7xg8y',
  'text': 'Cant say ive heard of one but this is a really neat idea!',
  'author': 'chasebencin',
  'score': 1},
 {'comment_id': 'ihmi7mf',
  'parent_id': 'ihmgyqq',
  'text': 'Will try have a look for this thanks',
  'author': 'Ainzi-RS',
  'score': 1},
 {'comment_id': 'ihmahmk',
  'parent_id': 'ihma1i4',
  'text': 'Ye I thought to if there is anything out there it would be a really good idea \nYou could bring items in from a inventory say move them around to suit then do it in real life \nJust like in the steam home',
  'author': 'Ainzi-RS',
  'score': 2},
 {'comment_id': 'ihmaqub',
  'parent_

In [63]:
print(len(unique_post_ids))
with open("unique_post_ids.txt", "w") as file:
    for item in unique_post_ids:
        file.write(item + "\n")

with open("unique_post_ids.txt", "r") as file:
    unique_post_ids_lst = [line.strip() for line in file]

# Print the list to verify
print(len(unique_post_ids_lst))

686
686


In [64]:
print(len(unique_comment_ids))
with open("unique_comment_ids.txt", "w") as file:
    for item in unique_comment_ids:
        file.write(item + "\n")

with open("unique_comment_ids.txt", "r") as file:
    unique_comment_ids_lst = [line.strip() for line in file]

# Print the list to verify
print(len(unique_comment_ids_lst))

16442
16442


In [65]:
with open("condensed_data.txt", "w") as file:
    json.dump(condensed_data, file)

In [66]:
with open("condensed_data.txt", "r") as file:
    condensed_data_lst = json.load(file)

In [67]:
condensed_data_lst[0]

[{'comment_id': 'w7xg8y',
  'parent_id': None,
  'text': 'Interior design',
  'author': 'Ainzi-RS',
  'score': 2},
 {'comment_id': 'ihmgyqq',
  'parent_id': 'w7xg8y',
  'text': "There was an IKEA app back when the Vive first came out. No idea if it is still a thing, but it's definitely a good idea...",
  'author': 'TheKramer89',
  'score': 2},
 {'comment_id': 'ihma1i4',
  'parent_id': 'w7xg8y',
  'text': 'Cant say ive heard of one but this is a really neat idea!',
  'author': 'chasebencin',
  'score': 1},
 {'comment_id': 'ihmi7mf',
  'parent_id': 'ihmgyqq',
  'text': 'Will try have a look for this thanks',
  'author': 'Ainzi-RS',
  'score': 1},
 {'comment_id': 'ihmahmk',
  'parent_id': 'ihma1i4',
  'text': 'Ye I thought to if there is anything out there it would be a really good idea \nYou could bring items in from a inventory say move them around to suit then do it in real life \nJust like in the steam home',
  'author': 'Ainzi-RS',
  'score': 2},
 {'comment_id': 'ihmaqub',
  'parent_

## Earlier (Aug 10)

In [ ]:
file_path = '/home/ubuntu/agents_ai/reddit_info_with_comments_postid_07242024b.json'
with open(file_path, 'r') as f:
    reddit_info = json.load(f)

In [4]:
posts_list = {}
i = 0
for key_primary, item_dict in reddit_info.items():
    for key_secondary, item_list in item_dict.items():
        for item in item_list:
            if item['Type'] == 'Post':
                i += 1
                posts_list[i] = item
print(f'***Posts found : {len(posts_list)}***')

***Posts found : 864***


In [5]:
casa_ai_descr = "Casa AI is the next generation tool for transforming interior spaces. Designed for homeowners, interior designers, \
and real estate professionals, our app uses advanced AI to reimagine any room based on your style and functional needs."

In [6]:
from langchain.llms import Ollama

In [8]:
llm = Ollama(model='llama3.1:8b')

In [9]:
post_ranker = Agent(
    role="Post Ranker",
    goal="Rank the top {n} posts that can be targeted for marketing of {product}",
    backstory="You will be given json data of reddit posts. You should"
               "identify the posts that will be relevant for the digital"
               "marketing of the {product}. You work is the basis for the topic identifier"
               "who will identify the topic of the top rated posts and comments.",
    allow_delegation=False,
	  verbose=True,
    llm=llm
)

In [10]:
ranked_post_format = "[1bzrhax','15tcfrx','14ueo0i'...]"

In [11]:
k = 10
ranking_task = Task(
    description=(
        "1. Prioritize the noteworthy posts from input "
        "   that will be relevant for marketing of {product}.\n"
        "2. Rank the top {n} posts.\n"
        "3. For output, include only the Rank calculated and 'Post_id' from json input.\n"
        "4. Input data to be used is {input_data}"
    ),
    expected_output="post_ids of top ranked posts in a python list format of {format}",
    agent=post_ranker)

In [12]:
crew = Crew(
    agents=[post_ranker],
    tasks=[ranking_task],
    verbose=2
)

In [ ]:
result = crew.kickoff(inputs={"product": casa_ai_descr, "n": k, "input_data": posts_list, "format": ranked_post_format})

In [15]:
result.raw

"[ '14u2b4o', '15tcfrx', '14ueo0i', '1bzrhax' ]\n\nNote: The post_ids are in a python list format as required by the problem."

In [ ]:
llm_llama31_70b = Ollama(model='llama3.1:70b')

In [ ]:
post_ranker_70b = Agent(
    role="Post Ranker",
    goal="Rank the top {n} posts that can be targeted for marketing of {product}",
    backstory="You will be given json data of reddit posts. You should"
               "identify the posts that will be relevant for the digital"
               "marketing of the {product}. You work is the basis for the topic identifier"
               "who will identify the topic of the top rated posts and comments.",
    allow_delegation=False,
	  verbose=True,
    llm=llm
)

In [ ]:
k = 10
ranking_task_70b = Task(
    description=(
        "1. Prioritize the noteworthy posts from input "
        "   that will be relevant for marketing of {product}.\n"
        "2. Rank the top {n} posts.\n"
        "3. For output, include only the Rank calculated and 'Post_id' from json input.\n"
        "4. Input data to be used is {input_data}"
    ),
    expected_output="post_ids of top ranked posts in a python list format of {format}",
    agent=post_ranker_70b)

In [ ]:
crew_70b = Crew(
    agents=[post_ranker_70b],
    tasks=[ranking_task_70b],
    verbose=2
)

In [ ]:
result_70b = crew.kickoff(inputs={"product": casa_ai_descr, "n": k, "input_data": posts_list, "format": ranked_post_format})